In [1]:
"""
Pointer Immediate Tasks:
- Run an experiment where I take trained pointer networks and transition them to a dominoe based value 
function and a gamma < 1, and show that they can learn to prioritize playing high value dominoes first. Then...
-  Add the context vector that encodes the number of turns left (with uncertainty?)
    - so the full pointer network will get an extra context input that describes how many turns are left
    - 0 rewards will be given after the possible turns are over
    - so the network will have to learn to get as much value out as quickly as possible
-  Also apply these networks to the vehicle routing problem?
-  Analyze encoding space of pointer networks...
-  Do the encoder swap of different pointer layers...
-  Does the speed of learning for the different networks on the sequencer task come from true performance or just sensitivity to the temperature? 
"""

"""
We can speed up some of the processing by doing all networks output in a list at once-- for example for reward computation. I think 
the sequencer is slower because measuring reward takes a long time...

Add mechanism for printing the arguments used to build a pointernetwork so the user can see what they did. 
Add mechanism for storing hidden parameters to entire pointer network

Add documentation of baseline updates and performance etc
Add some dataset specific summary plots and integrate into plotting code? 
Get the supervised learning methods working for each dataset and task
Checkpointing, figure making, logging, etc

it worked!!! now trying without embedding bias...
it works without embedding bias. It works (with different speeds per pointer layer!) with lower train temperature
 (but of course that could be because of differential sensitivity to temperature..., should test that directly)
now trying with 1 encoding layer. 

:)

TODO: 
DOMINOES SEQUENCER Comparison of max to real reward:
- Add target to batch (can do post-hoc, even if not requested)
- Measure reward of target
- Add a 2D vector comparing max and real reward for each batch element!!

TSP Distance Traveled:
- Explicitly measure the distance traveled by the agent in the TSP task
- Compare to Held-Karp Solution


TODO ASAP!!!!!!
I'm running experiments with attention only pointers and saving the networks. Will use trained networks
to test some other dataset specific variables and plots, etc when they're finished, then integrate those
into the main workflow. 

TODO ASAP!!!!!!
Make a checkpoint from the trained & saved dominoe_sequencer results to continue training etc.
"""

# put differences in parameters in logger!!!
# think more carefully about how to handle the ignore index for reward computation.....
# add special messages for curricula phases

# handling of checkpoints for curriculum learning is kinda rough...
# -- I think I can make a central system for all curricula that will handle all of this

# I need a smarter "load_experiment" update prms / exp.args method... (for example probably shouldn't update the checkpointing parameters)
# And in general need better analysis tools for rebuilding networks and rerunning data processing


# Interpretability Tools:
# https://github.com/TransformerLensOrg/TransformerLens?tab=readme-ov-file --- this has a really nice looking reading list...
# https://transformer-circuits.pub/2021/garcon/index.html

%reload_ext autoreload
%autoreload 2

# Imports
from time import time
from matplotlib import pyplot as plt
import torch
from ptrseq.experiments import get_experiment
from ptrseq.datasets import get_dataset
from ptrseq.experiments import get_experiment
from ptrseq.utils import get_scheduler, build_args, stack_results, get_dictionary_differences
from ptrseq.networks.net_utils import forward_batch

In [72]:
torch.set_grad_enabled(False)

experiment = "ptr_arch_comp"
args = dict(task="dominoe_sorter", encoder_method="attention", decoder_method="attention", replicates="3", embedding_bias="False")
exp = get_experiment(experiment, build=True, args=build_args(kvargs=args))

results = exp.load_experiment(use_saved_prms=False, verbose=True)
dataset = exp.prepare_dataset()

# input dimensionality
input_dim = dataset.get_input_dim()
context_parameters = dataset.get_context_parameters()

# create networks
nets, optimizers, prms = exp.create_networks(input_dim, context_parameters)
nets = exp.load_networks(nets)
for net in nets:
    net.eval()

# prepare dataset parameters
parameters = exp.make_train_parameters(dataset, train=False)
dominoes = dataset.get_dominoe_set(parameters["train"])

t = time()

# create example batch
batch = dataset.generate_batch(**(parameters | {"batch_size": 1024}))

# run data through the network
max_possible_output = parameters.get("max_possible_output")  # this is the maximum number of outputs ever
scores, choices = forward_batch(nets, batch, max_possible_output, temperature=1.0, thompson=False)

# measure rewards
rewards = [dataset.reward_function(choice, batch) for choice in choices]

print(time() - t)

1.2621960639953613
